In [ ]:
import Pkg
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.add("Hungarian")
Conda.add_channel("loopbio")
Conda.add("motmetrics")

using Conda
using Hungarian

In [2]:
function rect_min_max(r)
    x1y1 = r[:,1:2]
    x2y2 = x1y1.+r[:,3:end]
    x1y1,x2y2
end

function box_iou(o,h)
    o1, o2 = rect_min_max(o)
    h1, h2 = rect_min_max(h)
    
    o1 = reshape(o1,(size(o1)[1],1,size(o1)[2]))
    o2 = reshape(o2,(size(o2)[1],1,size(o2)[2]))
    h1 = reshape(h1,(1,size(h1)...))
    h2 = reshape(h2,(1,size(h2)...))
    
    i_min = max.(o1,h1)
    i_max = min.(o2,h2)
    
    i_size = max.(i_max.-i_min, 0)
    i_vol = i_size[:,:,1].*i_size[:,:,2]
    
    # Get volume of union.
    o_size = max.(o2 .- o1, 0)
    h_size = max.(h2 .- h1, 0)
    o_vol = o_size[:,:,1].*o_size[:,:,2]
    h_vol = h_size[:,:,1].*h_size[:,:,2]
    u_vol = o_vol .+ h_vol .- i_vol
    
    i_vol./u_vol
end

function iou_matrix(objs,hyps)
    if size(objs)[1] == 0 || size(hyps)[1] == 0
        return nothing
    end
    box_iou(objs,hyps)
end

keepdim(x) = reshape(x,1,size(x)[1])

keepdim (generic function with 1 method)

In [33]:
using DelimitedFiles

seqs = ["MOT17-02-DPM", "MOT17-02-SDP", "MOT17-02-FRCNN", "MOT17-04-DPM", "MOT17-04-SDP", "MOT17-04-FRCNN", "MOT17-05-DPM", "MOT17-05-SDP", "MOT17-05-FRCNN", "MOT17-09-DPM", "MOT17-09-SDP", "MOT17-09-FRCNN","MOT17-10-DPM", "MOT17-10-SDP", "MOT17-10-FRCNN", "MOT17-11-DPM", "MOT17-11-SDP", "MOT17-11-FRCNN", "MOT17-13-DPM", "MOT17-13-SDP", "MOT17-13-FRCNN"]
#seq = "MOT17-02-DPM"
for seq in seqs
    dets = readdlm("data/MOT17/train/"*seq*"/det/det.txt", ',', '\n')

    frames = unique(dets[:,1])

    active_tracks = []
    finished_tracks = []
    for frame in frames
        frame_dets = dets[dets[:,1].==frame,:]
        updated_tracks = []

        for track in active_tracks
            if size(frame_dets)[1] > 0
                ious = iou_matrix(keepdim(track[end][3:6]),frame_dets[:,3:6])
                bi = findmax(ious)[2][2]                # index of best frame_det
                push!(track,frame_dets[bi,:])           # assign det to track
                push!(updated_tracks, track)            # so we know which tracks were updated now
                frame_dets = frame_dets[1:end .!= bi,:] # delete the det
            end

        end

        # create new tracks
        if length(updated_tracks) != 0
            for updated_track in updated_tracks
                push!(active_tracks,updated_track)
            end
        end
        if size(frame_dets)[1] != 0
            for new_track in eachrow(frame_dets)
                push!(active_tracks,push!([],new_track[:]))
            end
        end
    end

    for active_track in active_tracks
        push!(finished_tracks, active_track)
    end

    # write IDs
    for ti in 1:size(finished_tracks,1)
        for di in 1:size(finished_tracks[ti],1)
            finished_tracks[ti][di][2] = ti
        end
    end
    
    # save results
    open(string("results/",seq,".txt"), "w") do io
        writedlm(io, vcat(finished_tracks),'\n')
    end
end